In [17]:
from datetime import datetime, timedelta
import heapq

class Task:
    def __init__(self, task_id, name, priority, duration, required_skills, due_date, dependencies=None):
        self.task_id = task_id
        self.name = name
        self.priority = priority
        self.duration = duration  # in hours
        self.required_skills = set(required_skills)
        self.due_date = due_date
        self.dependencies = dependencies if dependencies else []  # List of task IDs
        self.assigned_worker = None
        self.start_time = None
        self.end_time = None
        self.assigned_resource = None

class Worker:
    def __init__(self, worker_id, name, skills, availability):
        self.worker_id = worker_id
        self.name = name
        self.skills = set(skills)
        self.availability = sorted(availability)  # Sorted list of (start_time, end_time)

    def is_available(self, start_time, end_time):
        for i, (available_start, available_end) in enumerate(self.availability):
            if available_start <= start_time and available_end >= end_time:
                return i  # Return index of the available slot
        return -1

    def allocate_time(self, start_time, end_time, slot_index):
        """Adjust the worker's availability after task assignment."""
        available_start, available_end = self.availability.pop(slot_index)
        if available_start < start_time:
            self.availability.append((available_start, start_time))
        if available_end > end_time:
            self.availability.append((end_time, available_end))
        self.availability.sort()

class Resource:
    def __init__(self, resource_id, name, capacity):
        self.resource_id = resource_id
        self.name = name
        self.capacity = capacity  # Number of concurrent tasks it can handle
        self.availability = []  # Stores tuples of (start_time, end_time, allocated)

    def is_available(self, start_time, end_time):
        """Check if resource has available capacity in the given timeframe."""
        if not self.availability:
            return True
        for res_start, res_end, allocated in self.availability:
            if not (res_end <= start_time or res_start >= end_time) and allocated >= self.capacity:
                return False
        return True

    def allocate(self, start_time, end_time):
        """Allocate the resource within a time window."""
        self.availability.append((start_time, end_time, 1))
        self.availability.sort()

class TaskScheduler:
    def __init__(self):
        self.tasks = []
        self.workers = []
        self.resources = []
        self.scheduled_tasks = []

    def add_task(self, task):
        heapq.heappush(self.tasks, (-task.priority, task))  # Max heap based on priority

    def add_worker(self, worker):
        self.workers.append(worker)

    def add_resource(self, resource):
        self.resources.append(resource)

    def schedule_tasks(self):
        pending_tasks = {}  # Track tasks with unsatisfied dependencies
        finished_tasks = {}  # Track completed tasks with end times

        while self.tasks:
            _, task = heapq.heappop(self.tasks)

            # Ensure dependencies are completed
            if any(dep not in finished_tasks for dep in task.dependencies):
                pending_tasks[task.task_id] = task
                continue

            # Find the earliest worker who is available and has the right skills
            for worker in self.workers:
                earliest_start = max(datetime.now(), *[finished_tasks.get(dep, datetime.min) for dep in task.dependencies] or [datetime.now()])
                end_time = earliest_start + timedelta(hours=task.duration)
                slot_index = worker.is_available(earliest_start, end_time)

                if slot_index != -1 and worker.skills >= task.required_skills:
                    # Find an available resource if required
                    for resource in self.resources:
                        if resource.is_available(earliest_start, end_time):
                            task.assigned_worker = worker
                            task.start_time = earliest_start
                            task.end_time = end_time
                            task.assigned_resource = resource

                            worker.allocate_time(earliest_start, end_time, slot_index)
                            resource.allocate(earliest_start, end_time)

                            self.scheduled_tasks.append(task)
                            finished_tasks[task.task_id] = task.end_time  # Track when this task is done
                            break
                    break

        return self.scheduled_tasks

# Example Usage
task1 = Task(1, "Data Processing", 3, 4, ["Python"], datetime(2025, 2, 15))
task2 = Task(2, "Report Generation", 2, 2, ["SQL"], datetime(2025, 2, 16), dependencies=[1])

worker1 = Worker(1, "Alice", ["Python", "SQL"], [(datetime(2025, 2, 10), datetime(2025, 2, 20))])

resource1 = Resource(1, "Server A", capacity=2)

scheduler = TaskScheduler()
scheduler.add_task(task1)
scheduler.add_task(task2)
scheduler.add_worker(worker1)
scheduler.add_resource(resource1)

scheduled = scheduler.schedule_tasks()

if scheduled:
    for task in scheduled:
        print(f"Task {task.name} assigned to {task.assigned_worker.name} "
              f"using {task.assigned_resource.name} from {task.start_time} to {task.end_time}")
else:
    print("No tasks were scheduled.")


Task Data Processing assigned to Alice using Server A from 2025-02-10 21:42:27.130047 to 2025-02-11 01:42:27.130047
Task Report Generation assigned to Alice using Server A from 2025-02-11 01:42:27.130047 to 2025-02-11 03:42:27.130047
